Loading the packages:

In [ ]:
import os
import numpy as np
import scipy.linalg as lina
import scipy.optimize as opt
import pandas as pd
import matplotlib.pyplot as plt
import ast  # Module to parse string representations of dictionaries
import warnings
warnings.simplefilter('ignore')

# System Model

Initially, only the equation for one state (temperature or concentration) will be considered. For simplicity, the domain will be $[0,1]$, with Danckwerts boundary conditions:

$$\left\{\begin{array}{l} \dot{x} = D\partial_{\zeta\zeta} x -v\partial_{\zeta} x +kx\\
D\partial_\zeta x(0,t)-vx(0,t)=-Rv[x(1,t-\tau)+u(t-\tau_I)] \\
\partial_\zeta x(1,t)=0 \\
y(t)=x(1,t-\tau_O)
  \end{array}\right. $$

This model considers that the input is applied in the reactor's entrance, which is mixed with the recycle from the outlet. Input, output, and state delays are considered and represented by $\tau_I,\tau_O$, and $\tau$, respectively. 

Initializing system parameters:

In [ ]:
default_pars = {
    'k':10,
    'D':0.1,
    'v':0.5,
    'tau':1,
    'R':0.9,
    'label':'default'
}

## Eigenvalue Analysis

The eigenvalue problem, defined as $A\Phi(\zeta,\lambda)=\lambda\Phi(\zeta,\lambda)$, will result in the following system of equation for this system:

$$\left\{\begin{array}{l} \lambda\phi = D\partial_{\zeta\zeta} \phi -v\partial_{\zeta} \phi +k\phi\\
\lambda\phi_D=\dfrac{1}{\tau}\partial_{\zeta}\phi_D\\
D\partial_\zeta \phi(0)-v\phi(0)=-v\phi_D(0) \\
\partial_\zeta \phi(1)=0 \\
\phi_D(1)=\phi(1)\\
  \end{array}\right. $$

where $\Phi=[\phi,\,\phi_D]^T$, with $\phi$ as the state eigenfunction and $\phi_D$ as the eigenfunction related to the delay. By defining $X=[\phi,\, \partial_{\zeta}\phi,\,\phi_D]^T$, the following system of ODEs is obtained:

$$
\left\{\begin{array}{l}\partial_{\zeta}X=\begin{bmatrix} 0 & 1 & 0\\ \dfrac{\lambda-k}{D} & \dfrac{v}{D} & 0\\0 & 0 & \tau\lambda\end{bmatrix}X=ΛX \\
DX_2(0)-vX_1(0)=-RvX_3(0) \\
X_2(1)=0 \\
X_3(1)=X_1(1)\\ \end{array}\right.
$$

## Characteristic Equation

This is a system of first order ODE's, and the solution to such systems is given by:

$$ X(\zeta, \lambda) = e^{\Lambda \zeta} X (\zeta=0, \lambda) \\ \overset{\zeta = 1}{\Rightarrow} X(1, \lambda) = e^{\Lambda} X (\zeta=0) $$

Now, let's assume:

$$ e^{\Lambda} = Q(\lambda) = \begin{bmatrix} 
        q_{1} & q_{2} & q_{3} \\ q_{4} & q_{5} & q_{6} \\ q_{7} & q_{8} & q_{9}
    \end{bmatrix} $$


Thus, we may write:

$$\left\{\begin{array}{l}
X_1(1) = q_1 X_1(0) + q_2 X_2(0) + q_3 X_3(0) \\
X_2(1) = q_4 X_1(0) + q_5 X_2(0) + q_6 X_3(0) \\
X_3(1) = q_7 X_1(0) + q_8 X_2(0) + q_9 X_3(0)
\end{array}\right.$$

Now, we may go ahead and put the above expressions into boundary conditions to get the following:

$$\left\{\begin{array}{l}
Dx_2-vx_1=-Rvx_3 \\
q_4 x_1 + q_5 x_2 + q_6 x_3 = 0 \\
q_7 x_1 + q_8 x_2 + q_9 x_3 = q_1 x_1 + q_2 x_2 + q_3 x_3
\end{array}\right. \Rightarrow \left\{\begin{array}{l}
-vx_1 + Dx_2 - Rvx_3 = 0 \\
q_4 x_1 + q_5 x_2 + q_6 x_3 = 0 \\
(q_1 - q_7) x_1 + (q_2 - q_8) x_2 + (q_3 - q_9) x_3 = 0
\end{array}\right.$$


where $x_i$ is the same as $X_i(0)$.

For this particular case, we know that:

$$ q_{3} = q_{6} = q_{7} = q_{8} = 0 $$

This will further simplify the above system of equions into the following system:

$$\left\{\begin{array}{l}
-vx_1 + Dx_2 - Rvx_3 = 0 \\
q_4 x_1 + q_5 x_2 = 0 \\
q_1 x_1 + q_2 x_2 - q_9 x_3 = 0
\end{array}\right.$$

This is a $3 \times 3$ system of algebraic equations in the form of $\bar{A} \bar{x} = 0 $, with:

$$ \bar{A} = \begin{bmatrix}
-v & D & -Rv \\
q_4 & q_5 & 0 \\
q_1 & q_2 & -q_9
\end{bmatrix}; \quad \bar{x} = \begin{bmatrix}
x_1 \\ x_2 \\ x_3
\end{bmatrix} $$

 For such a system to have non-trivial solution (i.e. $\bar{x} \neq 0$), the dimension of the nullspace of the coefficients matrix $\bar{A}$ needs to be non-zero. This will happen if and only if the coefficients matrix $\bar{A}$ is rank-deficient. One way to make sure matrix $ \bar{A} $ is not full-rank, is to set a linear combination of its rows to be zero, with non-zero coefficients. Multiplying the second row of the matrix by $\alpha$ and the third row by $\beta$, we can write:

$$
\left\{\begin{array}{l}
-v + \alpha q_4 + \beta q_1 = 0 \\
D + \alpha q_5 + \beta q_2 = 0 \\
-Rv - \beta q_9 = 0 
\end{array}\right.
\Rightarrow
\left\{\begin{array}{l}
\alpha q_4 + \beta q_1 = v \\
\alpha q_5 + \beta q_2 = -D \\
\beta q_9 = -Rv 
\end{array}\right.
$$

The above system is a system of 3 equations and 3 unknowns (i.e. $\alpha$, $\beta$, and $\lambda$, with $\lambda$ being hidden in $q_i$ terms). By writing $\alpha$ and $\beta$ variables based on $q_i$ terms, we can get the characteristic equation.

Using the third equation, we can get:

$$ \beta = \frac{-Rv}{q_9} $$

Using the above equation to replace $\beta$ into the second equation will result in:

$$ \alpha = \frac{v}{q_4} \left(1 + \frac{R q_1}{q_9} \right) $$

Therefore, we can put the above expressions for $\alpha$ and $\beta$ into the first equation to get the characteristic equation, which is a non-linear function of the eigenvalue of the system, $\lambda$:

$$ f(\lambda) = D + v \frac{q_5}{q_4} \left( 1 + \frac{R q_1}{q_9} \right) - Rv \frac{q_2}{q_9} = 0 $$

We may now multiply both sides of the charactersitic equation by $q_4 q_9$ to avoid numerical errors while solfing for $f(\lambda)$. This will give:

$$ g(\lambda) = D q_4 q_9 + v [ q_5 q_9 + R (q_1 q_5 - q_2 q_4)] = 0 $$

## Numerical Solution

First we define a function to find the eigenvalues for the above characteristic equation. This function takes optional keyword argument as shown in the function docstring, returning a tuple with the following:

- solution_df: A pandas dataframe of all the solutions found based on the given arguments.
- par['label']: A string that briefly explains the parameters resulting in the obtained eigenvalue distribution.

P.S.: There is another function embeded inside the function below, that gives the value of the characteristic equation at each $\lambda$.

In [ ]:
def find_eig(**kwargs):
    """
    This function solves the char equation in complex plane for different initial guesses.

    Parameters:
        **kwargs (dict): Optional keyword arguments that can be passed to customize behavior.
            - par (dict):
                A complete set of parameters as a dictionary.
                default_pars = {
                    'k':-10,
                    'D':0.1,
                    'v':0.5,
                    'tau':1,
                    'R':0.9,
                    'label':'default'
                }
                If par is not passed, keys may be passed separately. Absent keys will take default values.
            - guess_single (complex):
                A single initial guess, complex number. Guess range will be ignored if this is passed.
            - guess_range_real ([float, float, int]):
                A list to create linspace over real axis, with the syntax [start, end, count]
            - guess_range_imag ([float, float, int]):
                A list to create linspace over imag axis, with the syntax [start, end, count]
            - tol_1 (float): tolerance to stop outer fsolve loop (passed directly to fsolve).
            - tol_2 (float): tolerance to stop inner fsolve loop (passed directly to fsolve).
            - tol_multiplier (float): to relax inner loop tolerance when saving the result.            

    Returns:
        pd.DataFrame: DataFrame of the solutions found with each solution in a row, containing the following columns:
            'Sol_r':    Real part of the obtained solution
            'Sol_i':    Imag part of the obtained solution
            'Guess_r':  Real part for the initial guess resulting in the obtained solution
            'Guess_i':  Imag part for the initial guess resulting in the obtained solution
            'g(x)':     Char equation value of the obtained solution
            'Label':    Label of the parameters leading to the obtained solution
            'Pars':     Complete pars dictionary leading to the obtained solution
            'kwargs':   Complete kwargs dictionary leading to the obtained solution
            the dataframe is sorted by 'Sol_r' column.
    """
    # Assign default values to missing keyword arguments for parameters
    if 'par' in kwargs:
        par = kwargs['par']
        if par['label'] == '':
            differing_pairs = {key: value for key, value in par.items() if value != default_pars[key] and key != 'label'}
            par['label'] = ', '.join([f"{key}: {value}" for key, value in differing_pairs.items()])
    else:
        par = default_pars.copy()
        for key in par:
            if key != 'label':
                par[key] = kwargs.get(key, par[key])
        if par != default_pars:
            differing_pairs = {key: value for key, value in par.items() if value != default_pars[key]}
            par['label'] = ', '.join([f"{key}: {value}" for key, value in differing_pairs.items()])
            
    # Assign default values to missing keyword arguments for initial guess values
    if not 'guess_single' in kwargs:
        guess_range_real = kwargs.get('guess_range_real', [-100,-100,1])
        guess_range_imag = kwargs.get('guess_range_imag', [5,5,1])
    else:
        guess_single_r = np.real(kwargs['guess_single'])
        guess_single_i = np.imag(kwargs['guess_single'])

        guess_range_real = [guess_single_r, guess_single_r, 1]
        guess_range_imag = [guess_single_i, guess_single_i, 1]
    
    # Assign default values to the rest of missing keyword arguments
    tol_1 = kwargs.get('tol_1', 1e-6)
    tol_2 = kwargs.get('tol_2', 1e-12)

    tol_multiplier = kwargs.get('tol_multiplier', 100)

    # Constructiong a dictionary to capture legit solutions
    solution_dict = {'Sol_r':[],'Sol_i':[],'Guess_r':[],'Guess_i':[],'g(x)':[], 'Label':[], 'Pars':[], 'kwargs':[]}

    # Constructiong a 2D mesh for different initial guess values
    mesh_builder = np.meshgrid(np.linspace(guess_range_real[0],guess_range_real[1],guess_range_real[2]),np.linspace(guess_range_imag[0],guess_range_imag[1],guess_range_imag[2]))
    mesh = mesh_builder[0] + mesh_builder[1] * 1j
    
    def char_eq(x):
        """
        This function evaluates the charachteristic equation at a given point.

        Parameters:
            x ([float, float]):
                A list of 2 elements, making up the complex eigenvalue to calculate char_eq.
        
        Returns:
            array[float, float]:
                A list of 2 elements, making up the complex value of char_eq at the given point.
        """
        l = complex(x[0], x[1])
        A = np.array([
            [0, 1, 0],
            [(l- par['k'])/par['D'], par['v']/par['D'], 0],
            [0, 0, par['tau'] * l]
        ])
        Q = lina.expm(A)
        q = np.insert(Q,0,0)
        y = par['D'] * q[4] * q[9] + par['v'] * (q[5] * q[9] + par['R'] * (q[1] + q[5] - q[2] * q[4]))
        return np.array([y.real, y.imag])

    for i in mesh:
        for m in i:
            m = np.array([m.real, m.imag])                      # obtaining an initial guess from the mesh as a complex number
            solution_array_initial = opt.fsolve(char_eq,m,xtol=tol_1)   # solving char_eq with a relaxed tol
            is_sol_initial = char_eq(solution_array_initial)                   # evaluationg the value of char_eq at the obtained relaxed solution
            is_sol_initial = (abs(complex(is_sol_initial[0],is_sol_initial[1])))
            # An inner loop seems to be necessary as sometimes the fsolve gives incorrect results that are ~+-2*pi from the radial complex answer of the real solution
            if is_sol_initial < tol_1 * tol_multiplier:
                solution_array = opt.fsolve(char_eq,solution_array_initial,xtol=tol_2)
                is_sol = char_eq(solution_array)                   # evaluationg the value of char_eq at the obtained relaxed solution
                is_sol = (abs(complex(is_sol[0],is_sol[1])))
                if np.isclose(is_sol,0,atol=tol_2*tol_multiplier):
                    solution_dict['Guess_r'].append(m[0])
                    solution_dict['Guess_i'].append(m[1])
                    solution_dict['g(x)'].append(is_sol)
                    solution_dict['Sol_r'].append(solution_array[0])
                    solution_dict['Sol_i'].append(solution_array[1])
                    solution_dict['Label'].append(par['label'])
                    solution_dict['Pars'].append(par)
                    solution_dict['kwargs'].append(kwargs)
                    solution_dict['Guess_r'].append(m[0])
                    solution_dict['Guess_i'].append(-m[1])
                    solution_dict['g(x)'].append(is_sol)
                    solution_dict['Sol_r'].append(solution_array[0])
                    solution_dict['Sol_i'].append(-solution_array[1])
                    solution_dict['Label'].append(par['label'])
                    solution_dict['Pars'].append(par)
                    solution_dict['kwargs'].append(kwargs)
                    continue
    
    solution_df = pd.DataFrame(solution_dict)
    solution_df = solution_df.sort_values(by=['Sol_r'])
    
    return (solution_df, par['label'])

We also define a handful of functions to write csv files of solution dataframes that have parameters' label as their metadata, and vice versa:

In [ ]:
def save_dataframe_with_metadata(df, metadata, output_filepath):
    # Write metadata as a comment in the CSV file
    with open(output_filepath, 'w') as f:
        f.write(f"# {metadata}\n")  # Write metadata line as a comment
        df.to_csv(f, index=False)  # Write the DataFrame

def read_dataframe_with_metadata(input_filepath):
    with open(input_filepath, 'r') as f:
        # Read the first line to get the metadata
        metadata_line = f.readline().strip()
        metadata = metadata_line.lstrip('# ').strip()
        
    # Read the remaining lines as DataFrame
    df = pd.read_csv(input_filepath, comment='#')
    
    return (df, metadata)

# Eigenvalue Distribution Analysis

In this step, we want to evaluate system's eigenvalues. Furthermore, we are interested in the effects of system parameters on its eigenvalue distribution. First, we obtain the eigenvalue distribution for a system with default parameters, a.k.a. the default system:

In [ ]:
# Solve and plot for default_par

Now we change the default parameters and define new parameter sets to obtain new systems. Then we call the solver function using the customized parameter values to see how changing each parameter will affect the eigenvalue distribution of the system:

In [ ]:
pars_list_k = []
pars_list_D = []
pars_list_tau = []
pars_list_R = []

par_list_list = [pars_list_k, pars_list_D, pars_list_tau, pars_list_R]
par_key_list = ['k', 'D', 'tau', 'R']

for i in np.linspace(0.75,1.25,3):
    for j in range(4):
        par = default_pars.copy()
        par[par_key_list[j]] = par[par_key_list[j]] * i
        par_list_list[j].append(par)

R = [-200,10,420]
I = [0,10,20]

c = 0
for P in par_list_list:
    for p in P:
        c += 1
        (df, title) = find_eig(par=p, guess_range_real=R, guess_range_imag=I)
        filename = str(c) + ".csv"
        df.to_csv(filename)

In [ ]:
folder_path = "CSV"  # Path to the folder containing the CSV files
dataframes = []

for i in range(1, 13):
    filename = os.path.join(folder_path, f"{i}.csv")
    df = pd.read_csv(filename)
    dataframes.append(df)

In [ ]:
# Create a 3x4 grid of subplots
fig, axes = plt.subplots(3, 4, figsize=(12, 8), sharex=True, sharey=True)

# Flatten the axes array for easy iteration
axes = axes.flatten()

for i, df in enumerate(dataframes):
    # Filter rows based on criteria
    filtered_df = df[(df['Sol_r'] > -200) & (df['Sol_i'] > -5) & (df['Sol_i'] < 5)]
    
    # Calculate the correct index for the subplot arrangement
    col = i // 3
    row = i % 3
    
    ax = axes[row * 4 + col]
    ax.scatter(filtered_df['Sol_r'], filtered_df['Sol_i'])
    
    # Parse the 'Pars' string representation back into a dictionary
    pars_dict = ast.literal_eval(df['Pars'][0])
    
    # Check if Pars dictionary matches default_pars
    if pars_dict == default_pars:
        title = 'Default'
    else:
        # Find key-value pairs that differ from default_pars
        differing_pairs = {key: value for key, value in pars_dict.items() if value != default_pars[key]}
        title = ', '.join([f"{key}: {value}" for key, value in differing_pairs.items()])
    
    ax.set_title(title)
    ax.set_xlabel('Sol_r')
    ax.set_ylabel('Sol_i')

    # Set axis limits to ensure visibility of x=0 and y=0 lines
    ax.axhline(0, color='black', linewidth=0.5)  # Horizontal line at y=0
    ax.axvline(0, color='black', linewidth=0.5)  # Vertical line at x=0
    ax.grid(True)

# Adjust layout to prevent overlapping labels and titles
plt.tight_layout()
plt.show()